# Parameters

In [ ]:
table_name <- "demux_BH3KTLDMXY"
experiment_name <- "gm12878_fresh_RNA"
papermill <- FALSE
mat.path <- NA

In [ ]:
papermill <- as.logical(papermill)

# Install Libraries
Install information can be found [here](https://satijalab.org/seurat/articles/install.html)\
Vignette [here](https://satijalab.org/seurat/articles/pbmc3k_tutorial.html)

In [ ]:
if (!requireNamespace("BiocManager", quietly = TRUE))
    install.packages("BiocManager", repos = "https://cran.r-project.org")
if (!requireNamespace("AnVIL", quietly = TRUE))
    BiocManager::install("AnVIL")
if (!requireNamespace("Seurat", quietly = TRUE))
    AnVIL::install("Seurat")
if (!requireNamespace("hdf5r", quietly = TRUE))
    install.packages("hdf5r")

In [ ]:
suppressMessages(library(AnVIL))
suppressMessages(library(hdf5r))
suppressMessages(library(Seurat))
suppressMessages(library(gridExtra))
plan("multicore", workers = 8)
set.seed(1234)

# Load Workspace Tables

In [ ]:
if (!papermill){
    table <- avtable(table_name)
}

In [ ]:
get_file <- function(path){
    dest <- getwd()
    gsutil_cp(path, dest)
    name <- basename(path)
    return(name)
}

if (!papermill){
    mat.path <- get_file(table$h5_matrix[table[, sprintf('%s_id', table_name)] == experiment_name])
}

# Generate UMAP

In [ ]:
data <- Read10X_h5(mat.path)
rna <- CreateSeuratObject(counts = data, project = "umap", min.cells = 3, min.features = 200)
rna

In [ ]:
dir.create("plots", showWarnings=F)
printPNG <- function(filename, plotObject, papermill, wf=1, hf=1){
	options(repr.plot.width = 7*wf, repr.plot.height = 7*hf)
	print(plotObject)
	if(papermill){
		png(sprintf("plots/%s.png", filename), width=480*wf, height=480*hf)
		print(plotObject)
		dev.off()
	}
}

# Calculate percent of mitochondrial reads
rna[["percent.mt"]] <- PercentageFeatureSet(rna, pattern = "^MT-")
obj <- VlnPlot(rna, features = c("nFeature_RNA", "nCount_RNA", "percent.mt"), ncol = 3)
printPNG('violinPlots', obj, papermill)

In [ ]:
plot1 <- FeatureScatter(rna, feature1 = "nCount_RNA", feature2 = "percent.mt")
plot2 <- FeatureScatter(rna, feature1 = "nCount_RNA", feature2 = "nFeature_RNA")

obj <- plot1 + plot2
printPNG('mitochondria', obj, papermill, wf=2)

In [ ]:
# Normalization
rna <- NormalizeData(rna, normalization.method = "LogNormalize", scale.factor = 10000)

In [ ]:
# Find Variable Features
rna <- FindVariableFeatures(rna, selection.method = "vst", nfeatures = 2000)

# Identify the 10 most highly variable genes
top10 <- head(VariableFeatures(rna), 10)

# plot variable features with and without labels
plot1 <- VariableFeaturePlot(rna)
plot2 <- LabelPoints(plot = plot1, points = top10, repel = TRUE)
obj <- plot1 + plot2
printPNG('features', obj, papermill, wf=2)

In [ ]:
# Scaling
all.genes <- rownames(rna)
rna <- ScaleData(rna, features = all.genes)

In [ ]:
# PCA
rna <- RunPCA(rna, features = VariableFeatures(object = rna))
print(rna[["pca"]], dims = 1:5, nfeatures = 5)
obj <- VizDimLoadings(rna, dims = 1:2, reduction = "pca")
printPNG('dimLoadings', obj, papermill, wf=1.5)

In [ ]:
obj <- DimPlot(rna, reduction = "pca")
printPNG('pca', obj, papermill)

In [ ]:
obj <- DimHeatmap(rna, dims = 1, cells = 500, fast= FALSE, balanced = TRUE)
printPNG('heatmap', obj, papermill)

In [ ]:
rna <- JackStraw(rna, num.replicate = 100)
rna <- ScoreJackStraw(rna, dims = 1:20)
obj <- JackStrawPlot(rna, dims = 1:15)
printPNG('jackstraw', obj, papermill)

In [ ]:
obj <- ElbowPlot(rna)
printPNG('elbow', obj, papermill)

In [ ]:
rna <- RunUMAP(rna, dims = 1:10)

In [ ]:
obj <- DimPlot(rna, reduction = "umap")
printPNG('umap', obj, papermill)